# GPT Model w/ Descriptions
Combined_Songs_Artists.csv

Model trained on Playlist_Names, Playlist_Descriptions, Songs and Artists combined in one string

In [ ]:
# Required Libraries
import pandas as pd
from datasets import Dataset
import ast
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments
import os
import csv
import torch

In [14]:
os.chdir('./DS6051-Project')
os.getcwd()

'/sfs/gpfs/tardis/home/nuf8ms/Documents/MSDS/LLM/DS6051-Project'

In [9]:
#df2 = pd.read_csv('DS6051-Project/Playlist_data_with_lyrics.csv')
#df3 = pd.read_csv('DS6051-Project/playlist_data.csv')
df = pd.read_csv('../Additional Data/Combined_Songs_Artists.csv')

In [10]:
# Load and parse the dataset

# Convert stringified lists to actual lists
# for col in ['Playlist_Songs', 'Playlist_Artists']:
#     df[col] = df[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

df["Playlist_Songs"] = df["Playlist_Songs"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

## Training Targets 1st Iteration, 3 epoch Loss ~ 3, 5 epoch Loss ~ 2.5
## playlist_data.csv
# Create prompt + output text for GPT-2 training
# def format_example(row):
#     song_lines = [f"{s} - {a}" for s, a in zip(row['Playlist_Songs'], row['Playlist_Artists'])]
#     return f"### Prompt: {row['Playlist_Name']}\n### Playlist:\n" + "\n".join(song_lines)

## Training Targets 2nd Iteration, 5 epoch Loss ~ 1.3
## playlist_data.csv
# def format_example(row):
#     lines = [f"[SONG] {s} [ARTIST] {a}" for s, a in zip(row["Playlist_Songs"], row["Playlist_Artists"])]
#     return f"### Prompt: {row['Playlist_Name']}\n### Playlist:\n" + "\n".join(lines)

## Training targets 3rd Iteration, 5 epoch Loss ~ 1.35
## playlist_data.csv
# def format_example(row):
#     lines = [f"[SONG] {song} [ARTIST] {artist}" for song, artist in zip(row["Playlist_Songs"], row["Playlist_Artists"])]
#     playlist_body = "\n".join(lines)
    
#     # Include Playlist_Description in the prompt for training only
#     return (
#         f"### Prompt: {row['Playlist_Name']}\n"
#         f"### Description: {row['Playlist_Description']}\n"
#         f"### Playlist:\n{playlist_body}"
#     )

## Training targets 4th Iteration, 
## Combined_Songs_Artists.csv
def format_example(row):
    # Use the song-artist strings as-is
    playlist_body = "\n".join(row["Playlist_Songs"])
    return (
        f"### Prompt: {row['Playlist_Name']}\n"
        f"### Description: {row['Playlist_Description']}\n"
        f"### Playlist:\n{playlist_body}"
    )

df['text'] = df.apply(format_example, axis=1)

# Convert to Hugging Face Dataset
dataset = Dataset.from_dict({"text": df['text'].tolist()})

In [11]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 doesn't have a pad token

model = GPT2LMHeadModel.from_pretrained("meta-llama/Llama-3.2-1B")
model.resize_token_embeddings(len(tokenizer))  # In case we add special tokens

# Tokenize
def tokenize(batch):
    encodings = tokenizer(batch["text"], padding="max_length", truncation=True, max_length=512)
    encodings["labels"] = encodings["input_ids"].copy()  # 🔥 Add this line
    return encodings
    
tokenized_dataset = dataset.map(tokenize, batched=True)
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

split_dataset = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

OSError: Can't load tokenizer for 'meta-llama/Llama-3.2-1B'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'meta-llama/Llama-3.2-1B' is the correct path to a directory containing all relevant files for a GPT2Tokenizer tokenizer.

### Test Script

### Test Script

In [6]:
training_args = TrainingArguments(
    output_dir="./Models/gpt2_Combined_Song_Artists_Eval_Checkpoints",
    overwrite_output_dir=True,
    eval_strategy='steps',
    eval_steps=100,
    num_train_epochs=20,
    per_device_train_batch_size=2,
    save_steps=500,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=50,
    fp16=True if torch.cuda.is_available() else False,
    load_best_model_at_end=True,
    metric_for_best_model='loss',
    greater_is_better=False
    
)

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_dataset
# )

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

trainer.train()

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss
100,2.913600,2.825726
200,2.797800,2.707207
300,2.666300,2.659324
400,2.666300,2.609694
500,2.456100,2.588639
600,2.525100,2.551381
700,2.365300,2.527956
800,2.510000,2.504852
900,2.717400,2.485724
1000,2.420800,2.462376


KeyboardInterrupt: 

In [12]:
model.save_pretrained("./Models/gpt2_Combined_Song_Artists_Eval")
tokenizer.save_pretrained("./Models/gpt2_Combined_Song_Artists_Eval")

('./models/gpt2_Combined_Song_Artists/tokenizer_config.json',
 './models/gpt2_Combined_Song_Artists/special_tokens_map.json',
 './models/gpt2_Combined_Song_Artists/vocab.json',
 './models/gpt2_Combined_Song_Artists/merges.txt',
 './models/gpt2_Combined_Song_Artists/added_tokens.json')

In [13]:
def generate_playlist(prompt, max_length=200):
    input_text = f"### Prompt: {prompt}\n### Playlist:\n"
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    output = model.generate(
        input_ids = input_ids.to('cuda'),
        max_length=max_length,
        temperature=0.9,
        top_p=0.95,
        do_sample=True,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id
    )

    result = tokenizer.decode(output[0], skip_special_tokens=True)
    return result.split("### Playlist:\n")[1].strip()

# Try it out!
prompt = "Road Trip with Classic Rock"
print(generate_playlist(prompt))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Light My Fire : INXS
Rocket Man (I Think It's Going To Be A Long Long Time) : Elton John
Black or White - Single Version : Michael Jackson
Moby Dick : Led Zeppelin
Carry on Wayward Son : Kansas
Hotel California - 2013 Remaster : Eagles
Dream On : Aerosmith
Layla : Derek & The Dominos
The Logical Song - Remastered 2012 : Neil Young
Johnny B. Goode - Single Version : Chuck Berry
More Than a Feeling : Boston
Vultaricious : War
Piano Man : Billy Joel
You've Got a Friend : James Taylor
Moby Dick : Led Zeppelin
Run to Me : Bryan Adams
Carry on Wayward Son : Kansas
Go Your Own Way - 2004 Remaster : Fleetwood Mac
Rock and Roll - Remaster : Led Zeppelin
Learning To Fly :


# LoRA Fine Tuning

In [8]:
import torch
import transformers
import argparse
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from peft import (
        get_peft_model, 
        prepare_model_for_kbit_training, 
        LoraConfig
    )
from trl import SFTTrainer

In [4]:
## Import Saved Model and use with generate_playlist function
model = GPT2LMHeadModel.from_pretrained("./Models/gpt2_Combined_Song_Artists", local_files_only = True)
tokenizer = GPT2Tokenizer.from_pretrained("./Models/gpt2_Combined_Song_Artists", local_files_only = True)

tokenizer.pad_token = tokenizer.eos_token

In [5]:
print(model)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=3072, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=1024)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=4096, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=4096)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=50257, bias=False)
)


In [9]:
lora_config = LoraConfig(
        r=8,
        lora_alpha=8,
        lora_dropout=0.1,
        target_modules=["c_attn", "c_proj", "attn_dropout", "resid_dropout"],
        bias="none",
        task_type="CAUSAL_LM",
    )

In [11]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

/sfs/gpfs/tardis/home/nuf8ms/Documents/MSDS/LLM/DS6051-Project/test/lib/python3.11/site-packages/peft/tuners/lora/layer.py:1768: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


ValueError: Target module Dropout(p=0.1, inplace=False) is not supported. Currently, only the following modules are supported: `torch.nn.Linear`, `torch.nn.Embedding`, `torch.nn.Conv1d`, `torch.nn.Conv2d`, `torch.nn.Conv3d`, `transformers.pytorch_utils.Conv1D`, `torch.nn.MultiheadAttention.`.